## **Principal Component Analysis & Kmeans Clustering**

#### Reflection exercises for each group:

1. At the top of the Colab notebook that you share with everyone, please include your responses to the following questions:
  - Choose at least 1 dataset to explore in more detail.
  - What is the projection for this dataset?
  - Where can you find more information on how the data were collected and how to interpret the metadata?
  - Think about what data type each variable is.
  - Is it vector or raster data? What properties exist for each dataset?
  - What resolution are your data?

2. At the top of the Colab notebook, write a short summary detailing the processing steps in the notebook and your results.
  - Although these topics may be far removed from your own interests, how could these steps and analyses help in your own work?

3. OPTIONAL - Expand your script by adding additional processing, analysis, or other data.

As you're working through your exercise, **add code chunks to further document your scripts. Add additional comments to the code itself to clarify complicated processes.**




---



In [ ]:
import numpy as np
import ee
import geemap
import geemap.colormaps as cm
ee.Authenticate()
ee.Initialize(project='')
geemap.ee_initialize(project='')

In [ ]:
############### PCA Helper Functions (from GEE documentation linked below) ###############
# From GEE: https://developers.google.com/earth-engine/guides/arrays_eigen_analysis

def get_principal_components(centered, scale, region):
  '''INPUTS:
  centered: a mean zero image,
  scale: int
  region: geometry over which to perform analysis
  RETURNS:
  P-band image, normalized by SD
  '''
  numBands = centered.bandNames().length()
  # Collapse bands into 1D array
  arrays = centered.toArray()

  # Compute the covariance of the bands within the region.
  covar = arrays.reduceRegion(
      reducer=ee.Reducer.centeredCovariance(),
      geometry=region,
      scale=scale,
      maxPixels=1e9,
  )

  # Get the 'array' covariance result and cast to an array.
  # This represents the band-to-band covariance within the region.
  covar_array = ee.Array(covar.get('array'))

  # Perform an eigen analysis and slice apart the values and vectors.
  eigens = covar_array.eigen()

  # This is a P-length vector of Eigenvalues.
  eigen_values = eigens.slice(1, 0, 1)
  # This is a PxP matrix with eigenvectors in rows.
  eigen_vectors = eigens.slice(1, 1)

  # Convert the array image to 2D arrays for matrix computations.
  array_image = arrays.toArray(1)

  # Left multiply the image array by the matrix of eigenvectors.
  principal_components = ee.Image(eigen_vectors).matrixMultiply(array_image)

  # Turn the square roots of the Eigenvalues into a P-band image.
  sd_image = (
      ee.Image(eigen_values.sqrt())
      .arrayProject([0])
      .arrayFlatten([get_new_band_names('sd', numBands)])
  )

  # Turn the PCs into a P-band image, normalized by SD.
  return (
      # Throw out an an unneeded dimension, [[]] -> [].
      principal_components.arrayProject([0])
      # Make the one band array image a multi-band image, [] -> image.
      .arrayFlatten([get_new_band_names('pc', numBands)])
      # Normalize the PCs by their SDs.
      .divide(sd_image)
  )

def get_new_band_names(prefix, numBands):
  seq = ee.List.sequence(1, numBands)
  return seq.map(lambda b: ee.String(prefix).cat(ee.Number(b).int()))

def PCA(maskedImage, scale, region):
  # image = maskedImage.unmask()
  image = maskedImage
  bandNames = image.bandNames()

  # Mean center the data to enable a faster covariance reducer
  # and an SD stretch of the principal components.
  meanDict = image.reduceRegion(**{
    'reducer': ee.Reducer.mean(),
    'geometry': region,
    'scale': scale,
    'maxPixels': 1e13,
    'tileScale': 16
  })
  means = ee.Image.constant(meanDict.values(bandNames))
  centered = image.subtract(means)

  return get_principal_components(centered, scale, region)

In [ ]:
### Import AOP data ###
# Import the AOP surface directional reflectance (SDR)
aopSDR = ee.ImageCollection('projects/neon-prod-earthengine/assets/DP3-30006-001')
# Import the AOP red-green-blue imagery (RGB)
aopRGB = ee.ImageCollection('projects/neon-prod-earthengine/assets/DP3-30010-001')


### FOR YOU TO FILL IN ###
mySR_data = aopSDR \
  .filterDate('2016-01-01', '2016-12-31') \
  .filterMetadata('NEON_SITE', 'equals', 'HARV') \
  .first()

myRGB_data = aopRGB \
  .filterDate('2016-01-01', '2016-12-31') \
  .filterMetadata('NEON_SITE', 'equals', 'HARV') \
  .first()

### View your selected AOP data ###
# Set up visualization params
RGB_bands = ['B053', 'B035', 'B019'] # These are the band names for the red, green, blue bands in the SDR data
rgbVis = {'min': 0, 'max': 255, 'gamma': 0.8} # This sets a nice range of values for mapping the RGB data
sdrVis = {'min': 0, 'max': 1260, 'gamma': 0.8} # This sets a nice range of values for mapping the RGB data

# Map
m = geemap.Map()
m.addLayer(mySR_data.select(RGB_bands), sdrVis, 'Surface reflectance')
m.addLayer(myRGB_data, rgbVis, 'RGB')
m.centerObject(myRGB_data, 12)

m




In [ ]:
############### Process the SDR image for input to PCA ###############
### Get a list of valid bands ###
# Select the WL_FWHM_B*** band properties (using regex)
properties = mySR_data.toDictionary()
wl_fwhm_dict = properties.select(['WL_FWHM_B+\d{3}'])

# Function to pull out the wavelength values only and convert the string to float
def get_wavelengths(x):
  str_split = ee.String(x).split(',')
  first_elem = ee.Number.parse((str_split.get(0)))
  return first_elem

# Function to pull out the band neames only
def get_bands(x):
  str_split = ee.String(x).split('_')
  return str_split.get(-1)

# Apply the function to the wavelength full-width-half-max list
wavelengths = np.array(wl_fwhm_dict.values().map(get_wavelengths).getInfo())
bands = np.array(wl_fwhm_dict.keys().map(get_bands).getInfo())

# Create list of bands that are not set to -100
# From metadata in SDR images "Wavelengths between 1340-1445nm and 1790-1955nm are set to -100"
masked_bands_indices = ((wavelengths>=1340) & (wavelengths<=1445)) | ((wavelengths >=1790) & (wavelengths <=1955))
valid_bands = bands[~masked_bands_indices]


### Create the reduced resolution SDR image for input to PCA ###
# Set params
SCALE = 300
SKIP_BANDS_NUM = 10

# Get list of valid bands to select (reducing from 426 bands to ~40)
bands_to_select = (
    mySR_data.select('B...')   # pull out reflectance bands
                  .bandNames().slice(0, -1, SKIP_BANDS_NUM)  # only select every 10th band
                  .getInfo()  # convert to python list
  )
bands_to_select = [b for b in bands_to_select if b in valid_bands] # filter out bad bands

# Create a raster where pixels=1 for <10% cloud cover and pixels=0 for >10% cloud cover. This is our mask.
clearWeather = mySR_data.select('Weather_Quality_Indicator').eq(1)

# Create lower spectral and spatial resolution image for input to PCA
mySR_data_coarse =  (
    mySR_data
          .select(bands_to_select) # This step is critical! If you leave the bad bands, you'll get an image where every pixels is nan!
          .reproject(mySR_data.projection().crs(), # reproject to 300m resolution
                     scale=SCALE)
          # .updateMask(jerc_clearWeather) # Optional
)

############### Run PCA ###############
pc_image = PCA(mySR_data_coarse, SCALE, mySR_data_coarse.geometry())


############### Map results ###############
RGBish_bands = ['B051', 'B031', 'B021']
RGBish_vis = {'bands': RGBish_bands, 'min': 0, 'max': 1700, 'opacity': 1.0, 'gamma': 1.0}

m = geemap.Map()
m.addLayer(mySR_data_coarse.select(RGBish_bands), RGBish_vis, 'rgb-ish coarse')
m.addLayer(mySR_data.select(RGBish_bands), RGBish_vis, 'rgb-ish')
m.addLayer(pc_image.select(['pc1', 'pc2', 'pc3']), {'bands': ['pc1', 'pc2', 'pc3'], 'min': -3, 'max': 3, 'opacity': 1.0, 'gamma': 1.0}, 'PCA')
m.centerObject(mySR_data, 12)
m

The clustering code below came from the [Google Earth Engine documentation](https://developers.google.com/earth-engine/guides/clustering), which is an amazing resource! As an **optional extension**, you can check out their example, which uses Landsat data to compare the clusters created with the AOP data to clusters created using Landsat data.

In [ ]:
# Define a region in which to generate a sample of the input.
region = mySR_data.geometry()

# Make the training dataset.
training = pc_image.select(['pc1', 'pc2', 'pc3']).sample(
  region=region,
  scale=SCALE,
  numPixels=5000
  )

# Instantiate the clusterer and train it.
clusterer_3 = ee.Clusterer.wekaKMeans(3).train(training)
clusterer_10 = ee.Clusterer.wekaKMeans(10).train(training)

# Cluster the input using the trained clusterer.
result_3 = pc_image.select(['pc1', 'pc2', 'pc3']).cluster(clusterer_3)
result_10 = pc_image.select(['pc1', 'pc2', 'pc3']).cluster(clusterer_10)

# Display the clusters with random colors.
m = geemap.Map()
m.addLayer(region, {}, 'ROI')
m.addLayer(mySR_data_coarse.select(RGBish_bands), RGBish_vis, 'rgb-ish coarse')
m.addLayer(mySR_data.select(RGBish_bands), RGBish_vis, 'rgb-ish')
m.addLayer(result_3.randomVisualizer(), {}, '3 clusters')
m.addLayer(result_10.randomVisualizer(), {}, '10 clusters')
m.centerObject(pc_image, 12)
m



---



## Additional Resources

* **End-to-End Google Earth Engine**: If you'd like to continue exploring the Earth Engine processes and applications, <a href="https://courses.spatialthoughts.com/end-to-end-gee.html#automatic-conversion-of-javascript-code-to-python" target="_blank"> SpatialThoughts Course - Ujaval Gandhi </a> has some nice examples you can follow.

